In [2]:
!pip install tvb-data --quiet
!pip install tvb-library==2.6 --quiet


In [23]:
# load necessary python modules
import numpy as np
import pandas as pd
import scipy
import os

# load TVB library
from tvb.simulator.lab import *
from tvb.basic.readers import FileReader

In [8]:
# set up the simulation

# select the SJHMR3D model as the local model and define its parameters
# for delta series choose 
oscilator = models.ReducedSetHindmarshRose(r=np.array([0.006]), a=np.array([1.0]), b=np.array([3.0]), c=np.array([1.0]), 
                                           d=np.array([5.0]), s=np.array([4.0]), 
                                           xo=np.array([-1.6]), K11=np.array([0.5]), K12=np.array([0.1]), K21=np.array([0.15]), 
                                           sigma=np.array([0.3]), 
                                           mu=np.array([2.2]), variables_of_interest=["xi","alpha"])
oscilator.configure()

In [39]:
# set up the structural connectivity
mypath="C:/Users/dinar/Desktop/50healthy_demo"  # insert the path to where all subjects connectivities are stored
subject="AA_20120815" # specify which subject to load

# load connectivity weights
reader = FileReader(file_path= mypath +'/' + subject +'/weights.txt')
w= reader.read_array(dtype=numpy.float64, skip_rows=0, use_cols=None, matlab_data_name=None)

# load region centers
rl_reader = FileReader(file_path=mypath +'/' + subject +'/labels.txt')    
c_reader = FileReader(file_path=mypath +'/' + subject +'/nodes.txt')    

rl= rl_reader.read_array( dtype=str, skip_rows=0, use_cols=(0,), matlab_data_name=None)
c= c_reader.read_array( dtype=numpy.float64, skip_rows=0, use_cols=(0,), matlab_data_name=None)
# centres = pd.read_csv(mypath +'/' + subject +'/centres.txt', header=None, index_col=None, sep='\t')
# rl= centres.iloc[:, 0].to_numpy()
# c= centres.iloc[:, 1:].to_numpy()

# load connectivity tract lengths
reader = FileReader(file_path=mypath +'/' + subject +'/tract_lengths.txt')
tl= reader.read_array(dtype=numpy.float64, skip_rows=0, use_cols=None, matlab_data_name=None)

# confige the connectivity
white_matter = connectivity.Connectivity(region_labels=rl, weights=w, tract_lengths=tl, centres=c)        


In [42]:
# specify the coupling function
# here a linear scaling function was used
# csf --> global coupling scaling factor
# is one of the parameters explored in this study
# for delta series it was varied from 0.05 to 0.25 in steps of 0.01
# for alpha series it was varied from 0.025 to 0.04 in steps of 0.001
# in this simulation one one value is chosen, but loop across all values to reproduce the findings of the paper
csf = 0.05
white_matter_coupling = coupling.Linear(a=np.array([csf]), b=np.array([0.0]))
white_matter_coupling.configure()

# specify a conduction speed to calculate time delays in the network 
# the 2nd parameter that was explored in this study
# for delta series it was varied from 20 to 100 in steps of 100
# for alpha series it was varied from 10 to 100 in steps of 10
# in this simulation one one value is chosen, but loop across all values to reproduce the findings of the paper
speed=100

# set up the integration scheme to solve the differential equations
# for delta series choose
heunint = integrators.HeunStochastic(dt=0.01220703125, noise=noise.Additive(nsig=np.array([1.0])))

# for alpha series uncomment the below code
# heunint = integrators.HeunStochastic(dt=0.05, noise=noise.Additive(nsig=array([0.001])))

In [43]:

# specify what data to record from the simulation using tvb monitory
# choose subsampling of the neural signal
p=3.90625 #<-- 256Hz
momo = monitors.SubSample(period=p)

# choose to generate BOLD signal using a hemodynamic response function
hrffunction=equations.MixtureOfGammas()
pb=500
mama = monitors.Bold(period=pb, hrf_kernel=hrffunction)

# put both monitors together
what_to_watch = (momo, mama)

# configure the simulation
sim = simulator.Simulator(model=oscilator, connectivity=white_matter,
                          coupling=white_matter_coupling, conduction_speed=speed,
                          integrator=heunint, monitors=what_to_watch)

sim.configure()


,value
Type,Simulator
conduction_speed,100.0
connectivity,Connectivity gid: 04c5b6f5-21dd-4ca3-ad2e-fbda4c9e2e4e
coupling,Linear gid: 0cbbe493-4e69-4955-b45a-723b1325c55e
gid,UUID('7107c937-5b30-4399-9a9a-50a62fb97787')
initial_conditions,None
integrator,HeunStochastic gid: 34cb26ec-6f93-4f4e-9a48-40c84cc4f0dd
model,ReducedSetHindmarshRose gid: 1a72de3e-ad33-42be-8790-058b0a07f102
monitors,"(, )"
simulation_length,1000.0


In [45]:
# specify simulation length
# for delta series choose
sim_length = 180000

# for alpha series uncomment the below code
# sim_length = 300000

# for alpha series 22min simulation length use
# sim_length = 1320000

In [ ]:
%%time

# perform the simulation
subs_data = []
subs_time = []
bold_data = []
bold_time = []

for subs, bold in sim(simulation_length=sim_length):
    if not subs is None:
        subs_time.append(subs[0])
        subs_data.append(subs[1])
    
    if not bold is None:
        bold_time.append(bold[0])
        bold_data.append(bold[1])


In [ ]:
# save the simulated time series
# declare a path where data should be saved
save_path="."
SUBS  = numpy.sum(numpy.array(subs_data),axis=3)
TSUBS = numpy.array(subs_time)
BOLD  = numpy.array(bold_data)
TBOLD = numpy.array(bold_time)
subsfile =save_path+'/'+subject+'csf_'+str(csf) +'_speed_' +str(speed)
scipy.io.savemat(subsfile, mdict={'subs_data': SUBS, 'subs_time': TSUBS ,'bold_data': BOLD, 'bold_time': TBOLD })
